# Caras lab ephys analysis pipeline
This pipeline is intended to be run after extracting behavioral timestamps and neuron spike times with our [MatLab pipeline](https://github.com/caraslab/caraslab-spikesortingKS2)

Files need to be organized in a specific folder structure or file paths need to be changed

File structure can be found in the Sample dataset folder

## Imports
Specific imports can be found within each function

In [1]:
%load_ext autoreload
%autoreload 2

from os import remove, makedirs
import warnings
from os.path import sep

from multiprocessing import Pool, cpu_count
from glob import glob

from helpers.run_ephys_pipeline import run_pipeline
from helpers.compile_fr_result_csv import compile_fr_result_csv
from matplotlib.pyplot import rcParams
from helpers.get_JSON_data import get_JSON_data
from helpers.PSTH_plotter_fromJSON import run_PSTH_pipeline
from auROC_analysis.auROC_heatmap_plotter import run_auROC_heatmap_pipeline
from helpers.recalculate_ePsych_responseLatency import recalculate_ePsych_responseLatency
from auROC_analysis.auROC_tslearnClustering_mp import run_ts_clustering

warnings.filterwarnings("ignore")

# Some plotting parameters
label_font_size = 11
tick_label_size = 7
legend_font_size = 6
line_thickness = 1

rcParams['figure.dpi'] = 600
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
rcParams['font.family'] = 'Arial'
rcParams['font.weight'] = 'regular'
rcParams['axes.labelweight'] = 'regular'

rcParams['font.size'] = label_font_size
rcParams['axes.labelsize'] = label_font_size
rcParams['axes.titlesize'] = label_font_size
rcParams['axes.linewidth'] = line_thickness
rcParams['legend.fontsize'] = legend_font_size
rcParams['xtick.labelsize'] = tick_label_size
rcParams['ytick.labelsize'] = tick_label_size
rcParams['errorbar.capsize'] = label_font_size
rcParams['lines.markersize'] = line_thickness
rcParams['lines.linewidth'] = line_thickness

## Set global paths and variables

FILE NAMING REQUIREMENTS

This pipeline matches files using filenames.

If you make changes to the filenaming structures you will have to edit the filename matching code

Here are the defaults that come out of the MatLab processing pipeline:
- Synapse:
    - Behavior file: SUBJ-ID-154_MML-Aversive-AM-210501-112033_trialInfo.csv
    - Spike time file: SUBJ-ID-154_210501_concat_cluster2627.txt
- Intan:
    - Behavior file: SUBJ-ID-231_2021-07-17_15-19-28_Active_trialInfo.csv
    - Spike time file: SUBJ-ID-231_210717_concat_cluster651.txt

If you need to alter the filename matching structure, edit this function: helpers.preprocess_files.find_spoutfile_and_breakpoint

In [3]:
DATA_PATH = '.' + sep + 'Sample_data'

SETTINGS_DICT = {
    'EXPERIMENT_TAG': 'OFCPL',  # Appends to start of summary files
    'SPIKES_PATH': DATA_PATH + sep + 'Spike times',
    'KEYS_PATH': DATA_PATH + sep + 'Key files',
    'OUTPUT_PATH': DATA_PATH + sep + 'Output',
    
    # If your data was concatenated before Kilosort
    'BREAKPOINT_PATH': DATA_PATH + sep + 'Breakpoints',
    
    # in seconds; for firing rate calculation to non-AM trials
    'NONAM_DURATION_FOR_FR': 0.9,
    
    # in seconds; for firing rate calculation to AM trials; shock artifact starts at ~0.9
    'TRIAL_DURATION_FOR_FR': {'Hit': 0.9, 'FA': 0.9, 'Miss': 0.9, 'Passive': 0.9},
    
    'AFTERTRIAL_FR_START': {'Hit': 1, 'FA': 1, 'Miss': 1.3, 'Passive': 1},  # Shock artifact is ~0.3s long
    'AFTERTRIAL_FR_END': {'Hit': 1.9, 'FA': 1.9, 'Miss': 2.2, 'Passive': 1.9},  # 0.9 duration to keep window consistent with trial
    
    'RESPTIME_FR_START': {'Hit': 0, 'FA': 0, 'Miss': 0.3, 'Passive': 0},  # Start of window following response. Irrelevant for passive
    'RESPTIME_FR_END': {'Hit': 1.9, 'FA': 1.9, 'Miss': 2.25, 'Passive': 1.9},  # 0.9 duration to keep window consistent with trial
    
    'BEFORERESP_FR_START': {'Hit': 0.5, 'FA': 0.5, 'Miss': 0.8, 'Passive': 0.5},  # Start of window preceding response. Irrelevant for passive
    'BEFORERESP_FR_END': {'Hit': 0, 'FA': 0, 'Miss': 0.3, 'Passive': 0},  # End of window preceding response
    
    # Unused for now but keep this period in mind when analyzing shocked misses
    'SHOCK_START_END': [0.95, 1.3],
    
    # These are for raw data extraction (no FR calculations)
    'PRETRIAL_DURATION_FOR_SPIKETIMES': 2,  # in seconds; for grabbing spiketimes around AM trials
    'POSTTRIAL_DURATION_FOR_SPIKETIMES': 5,  # in seconds; for grabbing spiketimes around AM trials
    
    'MULTIPROCESS': True,  # Turn off multiprocessing for easier debugging
    
    # For multiprocessing. Defaults to 4/5s of the number of cores
    'NUMBER_OF_CORES': 4 * cpu_count() // 5,

    # Only run these cells/subjects/sessions or None to run all
    'SESSIONS_TO_RUN': None,  # You can specify parts of the file name too
    'SESSIONS_TO_EXCLUDE': None,
    
    'OVERWRITE_PREVIOUS_CSV': True,  # False: appends to existing firing rate CSV file
    
    # Set up your recording platforms here
    'RECORDING_TYPE_DICT': {
        'SUBJ-ID-197': 'synapse',
        'SUBJ-ID-151': 'synapse',
        'SUBJ-ID-154': 'synapse',
        'SUBJ-ID-231': 'intan',
        # 'SUBJ-ID-232': 'intan',
        # 'SUBJ-ID-270': 'intan',
        'SUBJ-ID-389': 'intan',
        'SUBJ-ID-390': 'intan'
    },

    # Only needed for some older recordings processed by the MatLab pipeline. Newer files contain the sampling rates in them
    'SAMPLING_RATE_DICT': {
        'synapse': 24414.0625,
        'intan': 30000
    },
    
    # PSTH generation settings
    'PSTH_BIN_SIZE': 0.01,
    'PSTH_PRE_STIMULUS_DURATION': 2,
    'PSTH_POST_STIMULUS_DURATION': 4,
    'PSTH_FIXED_YLIM': 150,
    'PSTH_RASTER_YLIM': 30.5,  # ~Number of trials to display in the raster plot
    'PSTH_ALIGN_TO_RESPONSE': False,
    'PSTH_TRIALTYPES': [
        'Hit (shock)',  # Trials above threshold
        # 'Hit (no shock)', # Trials below threshold
        'False alarm',
        'Miss (shock)',  # Trials above threshold
        # 'Miss (no shock)', # Trials below threshold
        'Passive',
    ],
    
    #########################################
    # AuROC heatmap generation settings
    
    # Implemented auROC analyses types:
    # Trial-aligned types:
    #   TrialAligned_PassivePre_auroc, TrialAligned_Hit_shockFlagOn_auroc, TrialAligned_FA_auroc, 
    #   TrialAligned_Miss_shockFlagOn_auroc, TrialAligned_PassivePost_auroc, TrialAligned_GO_auroc, TrialAligned_GO_byAMdepth_auroc
    # Response-aligned types:
    #   ResponseAligned_Hit_auroc, ResponseAligned_Hit_shockFlagOn_auroc, ResponseAligned_FA_auroc, 
    #   ResponseAligned_Miss_shockOn_auroc, ResponseAligned_Hit_byAMdepth_auroc, ResponseAligned_Miss_shockOn_byAMdepth_auroc
    
    # If you want to implement a new one, simply tweak run_ephys_pipeline.py
    #########################################
    
    # This grouping file can contain any number of characteristics, such as cell type, response characteristics, clustering IDs etc. Has to be generated elsewhere; Otherwise set to None
    'AUROC_GROUPING_FILE': None, # DATA_PATH + sep + 'allUnits_list.csv',  # Or None
    'AUROC_GROUPING_VARIABLE': 'ActiveBaseline_modulation_direction',  # Or None
    'AUROC_UNIQUE_GROUPS': ['decrease', 'increase', 'none'],
    'AUROC_GROUP_COLORS': ['#E49E50', '#5AB4E5', '#939598'],
    
    'AUROC_TRIALTYPES': {
        # Use these if you would like to sort based on trial (AM)-aligned auROC
        'TrialAligned_PassivePre_auroc': [0, 1.5],
        'TrialAligned_Hit_shockFlagOn_auroc': [0, 1.5],
        'TrialAligned_FA_auroc': [0, 1.5],
        'TrialAligned_Miss_shockFlagOn_auroc': [1.4, 2.9],
        'TrialAligned_PassivePost_auroc': [0, 1.5],
        
        # Use these if you would like to sort based on response (spout offset)-aligned auROC
        # 'ResponseAligned_Hit_auroc': [-0.5, 0],
        # 'ResponseAligned_FA_auroc': [-0.5, 0],
        # 'ResponseAligned_Miss_shockOn_auroc': [-0.5, 0]
    },
    

    'SORT_BY_WHICH_TRIALTYPE': 'TrialAligned_Hit_shockFlagOn_auroc',
    
    'AUROC_BIN_SIZE': 0.1,
    'AUROC_PRE_STIMULUS_DURATION': 2,
    
    'AUROC_POST_STIMULUS_DURATION': 4,
    
    #########################################
    # Time series clustering parameters
    #########################################
    'MAXCLUSTERS': 10,  # maximum number of clusters to be evaluated (10)
    'BOOT_N': 50,  # number of iterations (maximum of 50 recommended due to computing time for DTW) (50)
    'SK_FACTOR': 2,  # error multiplication factor for choosing best number of clusters (2)
    'USE_GPU': False,  # If true, overrides the global multiprocessing flag; Not implemented yet
    'USE_TIBSHIRANI_CRITERION': True,  # If true, use f(k) ≥ f(k+1) - s{k+1}; if false, use the stricter f(k) + s{k} ≥ f(k+1) - s{k+1}
    'TS_TRIALTYPES': {
        # Use these if you would like to cluster based on trial (AM)-aligned auROC
        # 'TrialAligned_PassivePre_auroc': [0, 1.5],
        # 'TrialAligned_Hit_shockFlagOn_auroc': [0, 1.5],
        # 'TrialAligned_FA_auroc': [0, 1.5],
        # 'TrialAligned_Miss_shockFlagOn_auroc': [1.4, 2.9],
        # 'TrialAligned_PassivePost_auroc': [0, 1.5],
        
        # Use these if you would like to cluster based on response (spout offset)-aligned auROC
        'ResponseAligned_Hit_auroc': [0, 1.5],
        'ResponseAligned_FA_auroc': [0, 1.5],
        'ResponseAligned_Miss_shockFlagOn_auroc': [0.3, 1.8]
    },
    
    #########################################
    # Switchboard 
    # Below is a switchboard of functions you desire to run from the pipeline
    # If you change your mind later, you can just run the ones you want and the code will add it to existing JSON files
    #########################################
    'PIPELINE_SWITCHBOARD': {
        # Only relevant to AversiveAM task when recorded using older RPvds circuit
        # ATTENTION!! THIS FUNCTION WILL OVERWRITE THE RespLatency COLUMN IN YOUR EPSYCH FILES
        'recalculate_ePsych_responseLatency': True,
        
        # Trial-by-trial firing rates
        'firing_rate_to_trials': True,  
        
        # auROC analyses need by definition to combine trials, so each of these pool a specific type of trial and align to specific events
        # Trials aligned by AM onset
        'TrialAligned_Hit_auroc': True,
        'TrialAligned_Hit_shockFlagOn_auroc': True,

        'TrialAligned_FA_auroc': True,
        
        'TrialAligned_Miss_auroc': True,
        'TrialAligned_Miss_shockFlagOn_auroc': True,
        
        'TrialAligned_GO_auroc': True,  # Agnostic of whether miss or hit
        'TrialAligned_GO_byAMdepth_auroc': True,  # Agnostic of whether miss or hit
        
        # Response-aligned (by spoutOff events)
        'ResponseAligned_Hit_auroc': True,
        'ResponseAligned_Hit_shockFlagOn_auroc': True,

        'ResponseAligned_FA_auroc': True,
        
        'ResponseAligned_Miss_shockFlagOn_auroc': True,
        
        'ResponseAligned_Hit_byAMdepth_auroc': True,
        'ResponseAligned_Miss_shockFlagOn_byAMdepth_auroc': True,

        # PSTH plotting
        'plot_trialType_PSTH': True,
        'plot_AMDepth_PSTH': True
    }
    #########################################
}


## Calculate response latency using spoutInfo files
Only run this if your files come from an old (~2023 and earlier) RPvds circuit and did not record response latencies 


In [3]:
# Before processing each unit's files, check if user wants to recalculate response latencies
keys_paths = glob(SETTINGS_DICT['KEYS_PATH'] + sep + '*trialInfo.csv')
if SETTINGS_DICT['PIPELINE_SWITCHBOARD']['recalculate_ePsych_responseLatency']:
    recalculate_ePsych_responseLatency((keys_paths, SETTINGS_DICT))

Spout offset not registered properly in: .\Sample_data\Key files\SUBJ-ID-154_MML-Aversive-AM-210501-112033_trialInfo.csv
TrialID: 239


Spout offset not registered properly in: .\Sample_data\Key files\SUBJ-ID-154_MML-Aversive-AM-210509-151840_trialInfo.csv
TrialID: 171


Spout offset not registered properly in: .\Sample_data\Key files\SUBJ-ID-154_MML-Aversive-AM-210510-113653_trialInfo.csv
TrialID: 209


Spout offset not registered properly in: .\Sample_data\Key files\SUBJ-ID-154_MML-Aversive-AM-210512-154912_trialInfo.csv
TrialID: 70


Spout offset not registered properly in: .\Sample_data\Key files\SUBJ-ID-154_MML-Aversive-AM-210512-154912_trialInfo.csv
TrialID: 254


Spout offset not registered properly in: .\Sample_data\Key files\SUBJ-ID-154_MML-Aversive-AM-210514-145555_trialInfo.csv
TrialID: 14


Spout offset not registered properly in: .\Sample_data\Key files\SUBJ-ID-154_MML-Aversive-AM-210519-104756_trialInfo.csv
TrialID: 14




## Initial file matching then run the pipeline
Uses multiprocessing to process many units at once

In [5]:
makedirs(SETTINGS_DICT['OUTPUT_PATH'] + sep + 'JSON files', exist_ok=True)

# Load existing JSONs; will be empty if this is the first time running
json_filenames = glob(SETTINGS_DICT['OUTPUT_PATH'] + sep + 'JSON files' + sep + '*json')

# Clear older temp files if they exist
process_tempfiles = glob(SETTINGS_DICT['OUTPUT_PATH'] + sep + '*_tempfile_*.csv')
[remove(f) for f in process_tempfiles]

# Generate a list of inputs to be passed to each worker
input_lists = list()
memory_paths = glob(SETTINGS_DICT['SPIKES_PATH'] + sep + '*cluster*.txt')
keys_paths = glob(SETTINGS_DICT['KEYS_PATH'] + sep + '*trialInfo.csv')

for dummy_idx, memory_path in enumerate(memory_paths):
    if SETTINGS_DICT['SESSIONS_TO_RUN'] is not None:
        if any([chosen for chosen in SETTINGS_DICT['SESSIONS_TO_RUN'] if chosen in memory_path]):
            pass
        else:
            continue
            
    if SETTINGS_DICT['SESSIONS_TO_EXCLUDE'] is not None:
        if any([chosen for chosen in SETTINGS_DICT['SESSIONS_TO_EXCLUDE'] if chosen in memory_path]):
            continue
        else:
            pass

    if SETTINGS_DICT['MULTIPROCESS']:
        input_lists.append((memory_path, json_filenames, SETTINGS_DICT))
    else:
        run_pipeline((memory_path, json_filenames, SETTINGS_DICT))

if SETTINGS_DICT['MULTIPROCESS']:
    pool = Pool(SETTINGS_DICT['NUMBER_OF_CORES'])

    # # Feed each worker with all memory paths from one unit
    pool_map_result = pool.map(run_pipeline, input_lists)

    pool.close()

    pool.join()
    
compile_fr_result_csv(SETTINGS_DICT['EXPERIMENT_TAG'], SETTINGS_DICT['OUTPUT_PATH'], SETTINGS_DICT['OVERWRITE_PREVIOUS_CSV'])

KeyboardInterrupt: 

## Now we can use the JSONs exported from the code above for a faster exploration of the data

In [11]:
# Sessions you want to run; edit the specifics in helpers.get_JSON_data if you want to change
session_tags = ['pre', 'active', 'post', 'post1h']

# Load existing JSONs
json_filenames = glob(SETTINGS_DICT['OUTPUT_PATH'] + sep + 'JSON files' + sep + '*json')

# Retrieve data in JSON
data_dict = dict()
print("Loading data in JSONs...")
for session_type in session_tags:
    unit_list, data_list = (
        get_JSON_data(json_filenames, session_type, 
                        sessions_to_run=SETTINGS_DICT['SESSIONS_TO_RUN'], 
                        sessions_to_exclude=SETTINGS_DICT['SESSIONS_TO_EXCLUDE'])
    )
    
    for unit_idx, unit in enumerate(unit_list):
        if unit not in data_dict.keys():
            data_dict.update({unit: {}})
        data_dict[unit].update({session_type: data_list[unit_idx]})

Loading data in JSONs...


## PSTH plotting
Unsophisticated PSTH plotting engine. Need to make tweaks if you want something fancier

Uses multiprocessing to process many units at once

In [12]:
if SETTINGS_DICT['MULTIPROCESS']:
    pool = Pool(SETTINGS_DICT['NUMBER_OF_CORES'])

    # Feed each worker one unit_name
    input_list = [(unit_name, data_dict[unit_name], SETTINGS_DICT) for unit_name in data_dict.keys()]
    pool_map_result = pool.map(run_PSTH_pipeline, input_list)

    pool.close()

    pool.join()
else:
    [run_PSTH_pipeline((unit_name, data_dict[unit_name], SETTINGS_DICT)) for unit_name in data_dict.keys()]

# AuROC heatmaps
Quick summary of what auROC profiles look like across the entire population.

In [13]:
run_auROC_heatmap_pipeline(data_dict, SETTINGS_DICT)

# Time series clustering
This function calculates the optimal number of clusters from time series

It is optimal for time series because it uses dynamic time warping (DTW) as a distance metric

The optimal number of clusters is derived from an implementation of the gap-statistic by Tibshirani et al., 2001

Results are output as csv/pdf files

TODO: Add data to pre-existing JSON files maybe?


In [14]:
run_ts_clustering(data_dict, SETTINGS_DICT)

Running gap-stat clustering for metric ResponseAligned_Hit_auroc with 10 maxclusters and 50 simulations
Loading data in JSONs...
OptimalK start...
Clustering data using 1 cluster(s)...
Processing time: 0 min, 1.306 sec
Clustering data using 2 cluster(s)...
Processing time: 0 min, 0.020 sec
Clustering data using 3 cluster(s)...
Processing time: 0 min, 0.018 sec
Clustering data using 4 cluster(s)...
Processing time: 0 min, 0.026 sec
Clustering data using 5 cluster(s)...
Processing time: 0 min, 0.025 sec
Clustering data using 6 cluster(s)...
Processing time: 0 min, 0.037 sec
Clustering data using 7 cluster(s)...
Processing time: 0 min, 0.031 sec
Clustering data using 8 cluster(s)...
Processing time: 0 min, 0.030 sec
Clustering data using 9 cluster(s)...
Processing time: 0 min, 0.229 sec
Clustering data using 10 cluster(s)...
Processing time: 0 min, 0.232 sec
Initializing 12 process(es) for clustering random distributions in 50 simulations x 10 clusters = 500 iterations
Processing time: 0 

<Figure size 3000x3000 with 0 Axes>

<Figure size 3000x3000 with 0 Axes>

<Figure size 3000x3000 with 0 Axes>